In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

#PDFs in BDT and sindec?
import os

# set env flags to catch BLAS used for scipy/numpy 
# to only use 1 cpu, n_cpus will be totally controlled by csky
if False:
    os.environ['MKL_NUM_THREADS'] = "1"
    os.environ['NUMEXPR_NUM_THREADS'] = "1"
    os.environ['OMP_NUM_THREADS'] = "1"
    os.environ['OPENBLAS_NUM_THREADS'] = "1"
    os.environ['VECLIB_MAXIMUM_THREADS'] = "1"

import matplotlib as mpl
mpl.rcParams['figure.facecolor'] = 'w'
mpl.rcParams['savefig.facecolor'] = 'w'
import matplotlib.pyplot as plt
from matplotlib import colors, cm
import csky as cy
from csky import cext
import numpy as np
import astropy
#from icecube import astro
import histlite as hl
import healpy
import healpy as hp
import socket
import pickle
import copy
healpy.disable_warnings()
plt.rc('figure', facecolor = 'w')
plt.rc('figure', dpi=100)

## Define Settings

In [ ]:
selection_version = 'version-001-p01'

host_name = socket.gethostname()

if 'cobalt' in host_name:
    print('Working on Cobalts')
    plot_dir = '/data/user/mhuennefeld/data/analyses/DNNCascadeCodeReview/unblinding_checks/plots/unblinding/model_confusion_stacking'
    data_dir = os.path.join(plot_dir, 'data')
    
else:
    raise ValueError('Unknown host:', host_name)

In [ ]:
for dir_path in [plot_dir, data_dir]:
    if not os.path.exists(dir_path):
        print('Creating directory:', dir_path)
        os.makedirs(dir_path)

## Load Data

In [ ]:
repo = cy.selections.Repository()
specs = cy.selections.DNNCascadeDataSpecs.DNNC_10yr

In [ ]:
%%time

ana = cy.get_analysis(
    repo, selection_version, specs, 
    #gammas=np.r_[0.1:6.01:0.125],
)

In [ ]:
a = ana.anas[0]
a.sig

In [ ]:
a.bg_data

## Helpers

In [ ]:
from cycler import cycle
from copy import deepcopy

soft_colors = cy.plotting.soft_colors
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']


def get_bias_allt(tr, ntrials=200, n_sigs=np.r_[:101:10], quiet=False, poisson=False):
    trials = [
        (None if quiet else print(f'\r{n_sig:4d} ...', end='', flush=True))
        or
        tr.get_many_fits(ntrials, n_sig=n_sig, logging=False, seed=n_sig, poisson=poisson)
        for n_sig in n_sigs]
    if not quiet:
        print()
    for (n_sig, t) in zip(n_sigs, trials):
        t['ntrue'] = np.repeat(n_sig, len(t))
    allt = cy.utils.Arrays.concatenate(trials)
    return allt

def get_color_cycler():
    return cycle(colors)

def plot_ns_bias(ax, tr, allt, label=''):

    n_sigs = np.unique(allt.ntrue)
    dns = np.mean(np.diff(n_sigs))
    ns_bins = np.r_[n_sigs - 0.5*dns, n_sigs[-1] + 0.5*dns]
    expect_kw = dict(color='C0', ls='--', lw=1, zorder=-10)

    h = hl.hist((allt.ntrue, allt.ns), bins=(ns_bins, 100))
    hl.plot1d(ax, h.contain_project(1),errorbands=True, 
              drawstyle='default', label=label)
    lim = ns_bins[[0, -1]]
    ax.set_xlim(ax.set_ylim(lim))
    ax.plot(lim, lim, **expect_kw)
    ax.set_aspect('equal')

    ax.set_xlabel(r'$n_{inj}$')
    ax.set_ylabel(r'$n_s$')
    ax.grid()

def plot_gamma_bias(ax, tr, allt, label=''):

    n_sigs = np.unique(allt.ntrue)
    dns = np.mean(np.diff(n_sigs))
    ns_bins = np.r_[n_sigs - 0.5*dns, n_sigs[-1] + 0.5*dns]
    expect_kw = dict(color='C0', ls='--', lw=1, zorder=-10)
    expect_gamma = tr.sig_injs[0].flux[0].gamma

    h = hl.hist((allt.ntrue, allt.gamma), bins=(ns_bins, 100))
    hl.plot1d(ax, h.contain_project(1),errorbands=True, 
              drawstyle='default', label=label)
    lim = ns_bins[[0, -1]]
    ax.set_xlim(lim)
    ax.set_ylim(1, 4)
    ax.axhline(expect_gamma, **expect_kw)

    ax.set_xlabel(r'$n_{inj}$')
    ax.set_ylabel(r'$\gamma$')
    ax.grid()

def plot_bkg_trials(
            bg, fig=None, ax=None, 
            label='{} bg trials', 
            label_fit=r'$\chi^2[{:.2f}\mathrm{{dof}},\ \eta={:.3f}]$', 
            color=colors[0],
            density=False,
            bins=50,
        ):
    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 4))
    
    if density:
        h = bg.get_hist(bins=bins).normalize()
    else:
        h = bg.get_hist(bins=bins)
    if label is not None:
        label = label.format(bg.n_total)
    hl.plot1d(ax, h, crosses=True, color=color, label=label)

    # compare with the chi2 fit:
    if hasattr(bg, 'pdf'):
        x = h.centers[0]
        norm = h.integrate().values
        if label_fit is not None:
            label_fit = label_fit.format(bg.ndof, bg.eta)
        if density:
            ax.semilogy(x, bg.pdf(x), lw=1, ls='--', label=label_fit, color=color)
        else:
            ax.semilogy(x, norm * bg.pdf(x), lw=1, ls='--', label=label_fit, color=color)

    ax.set_xlabel(r'TS')
    if density:
        ax.set_ylabel(r'Density')
    else:
        ax.set_ylabel(r'number of trials')
    ax.legend()
        
    return fig, ax

## Setup Analysis

In [ ]:
import sys
sys.path.insert(0, '../..')

import config as cg

cg.base_dir = '/data/user/mhuennefeld/data/analyses/unblinding_v1.0.1_csky_bugfix_template_flux/'


In [ ]:
def get_gp_tr(template_str, cutoff=np.inf, gamma=None, cpus=20):
    cutoff_GeV = cutoff * 1e3
    gp_conf = cg.get_gp_conf(
        template_str=template_str, gamma=gamma, cutoff_GeV=cutoff_GeV, base_dir=cg.base_dir)
    tr = cy.get_trial_runner(gp_conf, ana=ana, mp_cpus=cpus)
    return tr

def get_catalog_tr(catalog, cutoff=np.inf, gamma=2.0, cpus=20):
    catalog_file = os.path.join(
        cg.catalog_dir, '{}_ESTES_12.pickle'.format(catalog))
    cat = np.load(catalog_file, allow_pickle=True)
    src = cy.utils.Sources(dec=cat['dec_deg'], ra=cat['ra_deg'], deg=True)
    cutoff_GeV = cutoff * 1e3

    conf = cg.get_ps_conf(src=src, gamma=gamma, cutoff_GeV=cutoff_GeV)
    tr = cy.get_trial_runner(ana=ana, conf=conf, mp_cpus=cpus)
    return tr

def get_combined_catalog_tr(cutoff=np.inf, gamma=2.0, cpus=20):
    dec_degs = []
    ra_degs = []
    for catalog in ['snr', 'pwn', 'unid']:
        catalog_file = os.path.join(
            cg.catalog_dir, '{}_ESTES_12.pickle'.format(catalog))
        cat = np.load(catalog_file, allow_pickle=True)
        dec_degs.append(cat['dec_deg'])
        ra_degs.append(cat['ra_deg'])
        
    dec_degs = np.concatenate(dec_degs)
    ra_degs = np.concatenate(ra_degs)
    
    src = cy.utils.Sources(dec=dec_degs, ra=ra_degs, deg=True)
    cutoff_GeV = cutoff * 1e3

    conf = cg.get_ps_conf(src=src, gamma=gamma, cutoff_GeV=cutoff_GeV)
    tr = cy.get_trial_runner(ana=ana, conf=conf, mp_cpus=cpus)
    return tr


#### Choose if to use bias-corrected values

In [ ]:
use_bias_correction = True

if use_bias_correction:
    stacking_catalog_gammas = {
        'snr': 2.735,
        'pwn': 2.99,
        'unid': 2.835,
        'combined_stacking': 2.81,
    }
else:
    stacking_catalog_gammas = {
        'snr': 2.75157378532275,
        'pwn': 2.9980144482315327,
        'unid': 2.846381577315917,
        'combined_stacking': 2.86,
    }

#### Get TrialRunners

In [ ]:
inj_tr_dict = {
    'fermibubbles_50TeV': get_gp_tr('fermibubbles', cutoff=50),
    'pi0': get_gp_tr('pi0'),
    'kra5': get_gp_tr('kra5'),
    'kra50': get_gp_tr('kra50'),
    'snr': get_catalog_tr('snr', gamma=stacking_catalog_gammas['snr']),
    'pwn': get_catalog_tr('pwn', gamma=stacking_catalog_gammas['pwn']),
    'unid': get_catalog_tr('unid', gamma=stacking_catalog_gammas['unid']),
    'combined_stacking': get_combined_catalog_tr(gamma=stacking_catalog_gammas['combined_stacking']),
}

tr_dict = inj_tr_dict

#### Get Results for each template

In [ ]:
res_dict = {}
for key in tr_dict.keys():
    if key in ['pi0', 'kra5', 'kra50']:
        f_path = os.path.join(
            cg.base_dir, 
            'gp/results/{}/{}_unblinded.npy'.format(key, key), 
        )
        res_dict[key] = np.load(f_path)
    elif key in ['fermibubbles_50TeV']:
        f_path = os.path.join(
            cg.base_dir, 'gp/results/fermibubbles/fermibubbles_unblinded.npy', 
        )
        total_trials = np.load(f_path)
        
        cutoffs = [50, 100, 500, np.inf]
        cutoff = float(key.replace('fermibubbles_', '').replace('TeV', ''))
        res_dict[key] = total_trials[np.searchsorted(cutoffs, cutoff)]
        
    elif key in ['snr', 'pwn', 'unid']:
        f_path = os.path.join(
            cg.base_dir, 
            'stacking/results/{}/{}_unblinded.npy'.format(key, key), 
        )
        res_dict[key] = np.load(f_path)

#### Define number of ns to inject

In [ ]:
from scipy.optimize import minimize
E0 = 100
unit = 1000

print('snr', tr_dict['snr'].to_ns(E2dNdE=6.2e-12, E0=E0, unit=unit))
print('pwn', tr_dict['pwn'].to_ns(E2dNdE=3.85e-12, E0=E0, unit=unit))
print('unid', tr_dict['unid'].to_ns(E2dNdE=4.7e-12, E0=E0, unit=unit))
print('pi0', tr_dict['pi0'].to_ns(E2dNdE=2.18e-11, E0=E0, unit=unit))

def get_ns_from_model_norm(template, model_norm, correction_factor=1.):
    def loss(ns):
        return (model_norm - tr_dict[template].to_model_norm(ns) * correction_factor)**2
    
    res = minimize(loss, x0=200)
    print(tr_dict[template].to_model_norm(res.x[0]) * correction_factor)
    return res.x[0]

print('kra5', get_ns_from_model_norm(template='kra5', model_norm=0.55))
print('kra50', get_ns_from_model_norm(template='kra50', model_norm=0.37))


In [ ]:
if use_bias_correction:
    ns_dict = {
        'pi0': 669.20,
        'kra5': 242.7,
        'kra50': 183.6,
        'snr': 189.5,
        'pwn': 247.3,
        'unid': 205.8,
        'combined_stacking': 335.,
        'fermibubbles_50TeV': 90., 
    }
else:
    ns_dict = {
        'pi0': 748.0433961026386,
        'kra5': 275.64561683178107,
        'kra50': 211.13586269951296,
        'snr': 218.59746844389832,
        'pwn': 279.6393079525785,
        'unid': 238.36706157922325,
        'combined_stacking': 335,
        'fermibubbles_50TeV': 95.94615440877195,
    }

#### Print Best Fit Fluxes

In [ ]:
if use_bias_correction:
    print('Showing bias-corrected fluxes')
for key, tr in tr_dict.items():
    if key in res_dict and not use_bias_correction:
        ns = res_dict[key][1]
    else:
        ns = ns_dict[key]
        if not use_bias_correction:
            print('No result found. Assuming ns of: {:3.1f}'.format(ns))
        
    dNdE = tr_dict[key].to_dNdE(ns=ns, E0=1e5)
    E2dNdE = tr_dict[key].to_E2dNdE(ns=ns, E0=100, unit=1e3)
    if not isinstance(E2dNdE, float):
        E2dNdE = E2dNdE[0]
    print('E2dNdE at 100 TeV in units of Tev {:3.3e} | {}'.format(E2dNdE, key))

#### Get bkg fits for each template

In [ ]:
bkg_file_dict = {
    'fermibubbles_50TeV': '{}/gp/trials/{}/{}/trials.dict'.format(cg.base_dir, 'DNNC', 'fermibubbles'),
    'pi0': '{}/gp/trials/{}/{}/trials.dict'.format(cg.base_dir, 'DNNC', 'pi0'),
    'kra5': '{}/gp/trials/{}/{}/trials.dict'.format(cg.base_dir, 'DNNC', 'kra5'),
    'kra50': '{}/gp/trials/{}/{}/trials.dict'.format(cg.base_dir, 'DNNC', 'kra50'),
    'snr': '{}/stacking/{}_bg.dict'.format(cg.base_dir, 'snr'),
    'pwn': '{}/stacking/{}_bg.dict'.format(cg.base_dir, 'pwn'),
    'unid': '{}/stacking/{}_bg.dict'.format(cg.base_dir, 'unid'),
    'combined_stacking': os.path.join(plot_dir, 'trials_combined_stacking.pkl'),
}
n_bkg_trials = 20000
seed = 1337

bkg_dict = {}
bkg_dist_dict = {}

for key, tr in tr_dict.items():
    if key in bkg_file_dict and os.path.exists(bkg_file_dict[key]):
        print('Loading background trials for template {}'.format(key))
        sig = np.load(bkg_file_dict[key], allow_pickle=True)
        if key in ['pi0', 'kra5', 'kra50']:
            bkg_dict[key] = sig['poisson']['nsig'][0.0]['ts']
        elif key in ['fermibubbles_50TeV']:
            cutoff = float(key.replace('fermibubbles_', '').replace('TeV', ''))
            print(key, 'cutoff', cutoff)
            bkg_dict[key] = sig['poisson']['cutoff'][cutoff]['nsig'][0.0]['ts']
        else:
            bkg_dict[key] = sig.ts
    
    else:
        print('Running background trials for template {}'.format(key))
        trials = tr.get_many_fits(
            n_trials=n_bkg_trials, seed=seed, mp_cpus=20)
        
        bkg_dict[key] = trials.ts
        
        out_file = os.path.join(plot_dir, 'trials_{}.pkl'.format(key))
        with open(out_file, 'wb') as f:
            pickle.dump(trials, f, protocol=2)
        

In [ ]:
def get_sigma(key):
    res = tr_dict[key].get_one_fit(TRUTH=True, flat=False)
    if key in bkg_dist_dict:
        bg = bkg_dist_dict[key]
    else:
        bg = cy.dists.Chi2TSD(bkg_dict[key])
    return bg.sf_nsigma(res[0])


In [ ]:
for k in tr_dict.keys():
    if k not in res_dict:
        res_ = tr_dict[k].get_one_fit(TRUTH=True, flat=False)
        res = {'ts': res_[0]}
        res.update(res_[1])
        res['sigma'] = get_sigma(k)
        res_dict[k] = res

#### Plot ts distribution

In [ ]:
for key, bg in bkg_dict.items():
    bg_tsd = cy.dists.TSD(bg)
    fig, ax = plot_bkg_trials(bg_tsd)
    if key in res_dict:
        ts = res_dict[key][0]
        ns = res_dict[key][1]
        ax.axvline(
            ts, color='0.8', ls='--', lw=2,
            label='TS: {:3.3f} | ns: {:3.1f}'.format(ts, ns), 
        )
    ts_5sig = bg_tsd.isf_nsigma(5)
    ax.axvline(
        ts_5sig, ls='--', lw=1,
        label='5-sigma TS: {:3.3f}'.format(ts_5sig), 
    )
    ax.set_title('Analysis: {}'.format(key))
    ax.set_yscale('log')
    ax.legend()
    fig.savefig('{}/ts_dist_{}.png'.format(plot_dir, key))

#### Get trials for each template

In [ ]:
from tqdm.notebook import tqdm_notebook as tqdm

n_trials = 1000
seed = 42
trials_dict = {}

for key, tr in inj_tr_dict.items():
    
    n_sig = ns_dict[key]
    trials = []
    
    print('Injecting {} signal events for template {}'.format(n_sig, key))
    for i in tqdm(range(n_trials), total=n_trials):
        trials.append(tr.get_one_trial(n_sig=n_sig, poisson=True, seed=seed + i))
    
    trials_dict[key] = trials
    

#### Get fits for each template combination

In [ ]:
from multiprocessing import Pool

n_cpus = 20
recompute = False

ts_dict = {}
for key_inj, tr_inj in inj_tr_dict.items():
    
    for key, tr in tr_dict.items():
        
        # define output file
        out_file = '{}/inj_{}_fit_{}.pkl'.format(data_dir, key_inj, key)
        
        if not os.path.exists(out_file) or recompute:
            print('Computing TS values for injection {} and testing with {}'.format(key_inj, key))

            ts_values = []
            n_values = len(trials_dict[key_inj])

            if n_cpus > 1:
                print('Running pool with {} cpus'.format(n_cpus))
                def compute_trial(i):
                    return tr.get_one_fit_from_trial(trials_dict[key_inj][i])

                with Pool(n_cpus) as p:
                    ts_values_i = list(tqdm(p.imap(compute_trial, range(len(trials_dict[key_inj]))), total=n_values))
                ts_values.extend(ts_values_i)
                p.close()
            else:
                for trial in tqdm(trials_dict[key_inj], total=n_values):
                    ts_values.append(tr.get_one_fit_from_trial(trial)) 

            ts_values = np.array(ts_values)
            
            with open(out_file, 'wb') as f:
                pickle.dump(ts_values, f, protocol=2)
        else:
            print('Loading TS values for injection {} and testing with {}'.format(key_inj, key))
            with open(out_file, 'rb') as handle:
                ts_values = pickle.load(handle)
        
        ts_dict[(key_inj, key)] = cy.utils.Arrays({
          'ts': ts_values[:, 0],  
          'ns': ts_values[:, 1],  
        })



#### Compute Significance

In [ ]:
p_val_dict = {}
sigma_dict = {}

for key, ts_values in tqdm(ts_dict.items(), total=len(ts_dict)):
    if len(bkg_dict[key[1]]) < 100000:
        if key[1] in bkg_dist_dict:
            bg = bkg_dist_dict[key[1]]
        else:
            bg = cy.dists.Chi2TSD(bkg_dict[key[1]])
            bkg_dist_dict[key[1]] = bg
    else:
        bg = cy.dists.TSD(bkg_dict[key[1]])
    max_bg_ts = np.max(bg.values)
    mask_above = ts_values.ts > max_bg_ts
    ts = np.array(ts_values.ts)
    if np.sum(mask_above) > 0:
        print('Setting {} ts values to max bkg ts value of {}.'.format(
            np.sum(mask_above), max_bg_ts))
        ts[mask_above] = max_bg_ts
    p_val_dict[key] = bg.sf(ts)
    sigma_dict[key] = bg.sf_nsigma(ts)


#### Save/Load Values

In [ ]:
if use_bias_correction:
    res_file = os.path.join(plot_dir, 'trials_and_fits_bias_corrected.pkl')
else:
    res_file = os.path.join(plot_dir, 'trials_and_fits.pkl')


if not os.path.exists(res_file):
    print('Writing results to file')
    with open(res_file, 'wb') as f:
        results = {
            'p_val_dict': p_val_dict,
            'sigma_dict': sigma_dict,
            'ts_dict': ts_dict,
            'res_dict': res_dict,
            #'trials_dict': trials_dict,
            #'tr_dict': tr_dict,
        }
        pickle.dump(results, f, protocol=-1)

if True:
    print('Loading results from file')
    with open(res_file, 'rb') as handle:
        results = pickle.load(handle)
    p_val_dict = results['p_val_dict']
    sigma_dict = results['sigma_dict']
    ts_dict = results['ts_dict']
    

#### Plot Distributions

In [ ]:
from itertools import cycle

def plot_model_confusion(inj_keys, testing_keys=None, bins=np.linspace(0, 6, 20)):
    n_keys = len(inj_keys)
    
    if testing_keys is None:
        testing_keys = inj_keys
        
    fig, axes = plt.subplots(n_keys, 1, figsize=(12, 4*n_keys), sharex=True)
    
    if n_keys <= 5:
        color_cycle = cycle(soft_colors)
    else:
        color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

    for i, key_inj in enumerate(inj_keys):

        ax = axes[i]
        ax.set_title('Injection: {}'.format(key_inj))

        for j, key in enumerate(testing_keys):
            sigmas = sigma_dict[(key_inj, key)]
            if not np.isfinite(sigmas).all():
                raise ValueError(sigmas)
            color = next(color_cycle)
            
            if key in ['pi0', 'kra5', 'kra50', 'fermibubbles_50TeV']:
                unblinded_ts = res_dict[key][3]
            elif key in ['snr', 'pwn', 'unid']:
                unblinded_ts = res_dict[key][5]
            else:
                unblinded_ts = np.nan
            
            ax.hist(
                sigmas, bins=bins, 
                histtype='step',
                color=color,
                label=(
                    r'Testing: {}'.format(key) + ' [$\sigma_{50\%}$ = ' 
                    + '{:3.2f}'.format(np.median(sigmas))
                    + ' | $\sigma_\mathrm{unblinded}$ = ' 
                    + '{:3.2f}]'.format(unblinded_ts)
                ),
            )
            ax.axvline(np.median(sigmas), color=color, ls='--')
            ax.axvline(unblinded_ts, color=color, ls='-.')

        ax.plot(np.inf, np.inf, color='0.7', ls='--', label='Median $\sigma$')
        ax.plot(np.inf, np.inf, color='0.7', ls='-.', label='Unblinded $\sigma$')
        ax.legend(loc='upper left')
        ax.set_ylabel('Frequency')

    axes[-1].set_xlabel('Significance in $\sigma$')

    fig.tight_layout()
    return fig


fig = plot_model_confusion(inj_keys=list(tr_dict.keys()), testing_keys=list(tr_dict.keys()))
fig.savefig('{}/model_confusion_all.png'.format(plot_dir))

fig = plot_model_confusion(inj_keys=['pi0', 'kra5', 'kra50'], testing_keys=['combined_stacking', 'snr', 'pwn', 'unid'])
fig.savefig('{}/model_confusion_stacking_combined.png'.format(plot_dir))

fig = plot_model_confusion(inj_keys=['combined_stacking', 'fermibubbles_50TeV'], testing_keys=['pi0', 'kra5', 'kra50', 'combined_stacking', 'fermibubbles_50TeV'])
fig.savefig('{}/model_confusion_injected_combined_stacking_fermibubbles.png'.format(plot_dir))

fig = plot_model_confusion(inj_keys=['combined_stacking', 'pi0'], testing_keys=['pi0', 'snr', 'pwn', 'unid', 'combined_stacking'])
fig.savefig('{}/model_confusion_combined_stacking.png'.format(plot_dir))

fig = plot_model_confusion(inj_keys=['fermibubbles_50TeV', 'pi0'], testing_keys=['pi0', 'kra5', 'kra50', 'snr', 'pwn', 'unid', 'fermibubbles_50TeV'])
fig.savefig('{}/model_confusion_fermibubbles.png'.format(plot_dir))

fig = plot_model_confusion(inj_keys=['pi0', 'kra5', 'kra50'], testing_keys=['pi0', 'kra5', 'kra50', 'snr', 'pwn', 'unid', 'fermibubbles_50TeV'])
fig.savefig('{}/model_confusion_gp_inj.png'.format(plot_dir))

fig = plot_model_confusion(inj_keys=['pi0', 'kra5', 'kra50'], testing_keys=['snr', 'pwn', 'unid'])
fig.savefig('{}/model_confusion_stacking.png'.format(plot_dir))

fig = plot_model_confusion(inj_keys=['pi0', 'kra5', 'kra50'], testing_keys=list(tr_dict.keys()))
fig.savefig('{}/model_confusion_stacking_all.png'.format(plot_dir))

fig = plot_model_confusion(inj_keys=['snr', 'pwn', 'unid'], testing_keys=['pi0', 'kra5', 'kra50'])
fig.savefig('{}/model_confusion_stacking__stacking.png'.format(plot_dir))

fig = plot_model_confusion(inj_keys=['snr', 'pwn', 'unid'], testing_keys=list(tr_dict.keys()))
fig.savefig('{}/model_confusion_stacking__stacking_all.png'.format(plot_dir))

fig = plot_model_confusion(inj_keys=['snr', 'pwn', 'unid'], testing_keys=['snr', 'pwn', 'unid'])
fig.savefig('{}/model_confusion_stacking__stacking_confusion.png'.format(plot_dir))


#### Plot matrix of median values

In [ ]:
import matplotlib.patheffects as pe

def get_median_sigma_matrix(model_keys=list(tr_dict.keys())):
    n_keys = len(model_keys)
    matrix_diff = np.zeros((n_keys, n_keys))

    for i, key_inj in enumerate(model_keys):
        for j, key in enumerate(model_keys):

            sigmas = sigma_dict[(key_inj, key)]
            if not np.isfinite(sigmas).all():
                raise ValueError(sigmas)

            if key in ['pi0', 'kra5', 'kra50', 'fermibubbles_50TeV']:
                unblinded_sigma = res_dict[key][3]
            elif key in ['snr', 'pwn', 'unid']:
                unblinded_sigma = res_dict[key][5]
            elif 'sigma' in res_dict[key]:
                unblinded_sigma = res_dict[key]['sigma']
            else:
                unblinded_sigma = np.nan
            
            sigma_diff = unblinded_sigma - np.median(sigmas)
            matrix_diff[i, j] = sigma_diff
    return matrix_diff, model_keys

def plot_sigma_matrix(
            matrix, model_keys,
            bound=2,
            name_converter={}, value_str=r'{:+.1f}$\sigma$',
            cmap=plt.cm.RdBu,
        ):
    fig, ax = plt.subplots(figsize=(6, 4))

    model_names = [name_converter.get(m, m) for m in model_keys]

    # extract all colors from the .jet map
    cmaplist = [cmap(i) for i in range(cmap.N)]
    # force the first color entry to be grey
    # cmaplist[0] = (.5, .5, .5, 1.0)

    # create the new map
    cmap = mpl.colors.LinearSegmentedColormap.from_list(
        'Custom cmap', cmaplist, cmap.N)

    # define the bins and normalize
    bounds = np.linspace(-bound, bound, 50)
    norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

    image = ax.imshow(matrix, interpolation='none', norm=norm, cmap=cmap)

    # Loop over data dimensions and create text annotations.
    for i in range(len(model_keys)):
        for j in range(len(model_keys)):
            color_float = np.clip(np.abs(matrix[i, j])/ (bound), 0, 1.)
            color_float = (color_float - 0.5)**0.1 + 0.5
            color_float =  np.clip(color_float, 0., 1.)
            if not np.isfinite(color_float):
                color_float = 0.

            color = '{:.2f}'.format(color_float)
            text = ax.text(
                j, i, value_str.format(matrix[i, j]),
                ha="center", va="center", color=color, fontsize=7,
            )

    cbar = plt.colorbar(image, ax=ax)
    cbar.set_label(
        r'$\Delta \sigma = \sigma_\mathrm{unblinded} - \sigma_{50\%}$')
    plt.xticks(range(len(model_keys)), model_names, fontsize=8, rotation=90)
    plt.yticks(range(len(model_keys)), model_names, fontsize=8)
    ax.set_xlabel('Tested Model')
    ax.set_ylabel('Injected Model')
    return fig, ax, cbar

name_converter = {
    'fermibubbles_50TeV': r'FB$_{50}$',
    'pi0': r'$\pi^0$',
    'kra5': r'KRA$_\gamma^{5}$',
    'kra50': r'KRA$_\gamma^{50}$',
    'snr': r'SNR',
    'pwn': r'PWN',
    'unid': r'UNID',
    'combined_stacking': r'All S.',
}

chosen_models = ['fermibubbles_50TeV', 'pi0', 'kra5', 'kra50', 'snr', 'pwn', 'unid', 'combined_stacking']
chosen_models = ['fermibubbles_50TeV', 'pi0', 'kra5', 'kra50', 'snr', 'pwn', 'unid']
matrix, model_keys = get_median_sigma_matrix(model_keys=chosen_models)
fig, ax, cbar = plot_sigma_matrix(matrix, model_keys=model_keys, name_converter=name_converter)
#ax.set_title('Model Confusion Test')
fig.tight_layout()
fig.savefig('{}/model_confusion_matrix.png'.format(plot_dir))
matrix

#### Plot P-value Matrix

In [ ]:
def get_pval_matrix(model_keys=list(tr_dict.keys())):
    n_keys = len(model_keys)
    matrix_diff = np.zeros((n_keys, n_keys))

    for i, key_inj in enumerate(model_keys):
        for j, key in enumerate(model_keys):

            sigmas = sigma_dict[(key_inj, key)]
            if not np.isfinite(sigmas).all():
                raise ValueError(sigmas)

            if key in ['pi0', 'kra5', 'kra50', 'fermibubbles_50TeV']:
                unblinded_sigma = res_dict[key][3]
            elif key in ['snr', 'pwn', 'unid']:
                unblinded_sigma = res_dict[key][5]
            elif 'sigma' in res_dict[key]:
                unblinded_sigma = res_dict[key]['sigma']
            else:
                unblinded_sigma = np.nan
            
            # do 2-sided test here:
            sorted_sigmas = np.sort(sigmas)
            idx = np.searchsorted(np.sort(sigmas), unblinded_sigma)
            fraction = float(idx)/len(sigmas)
            if fraction < 0.5:
                pval = 2 * fraction
            else:
                pval = 2 * (1 - fraction)
            matrix_diff[i, j] = pval
    return matrix_diff, model_keys

def plot_matrix_pval(
            matrix, model_keys,
            bound=-1.0,
            name_converter={},
            value_str=r'{:.3f}',
            lower_bound=0.001,
            cmap=plt.cm.cividis_r,
        ):
    fig, ax = plt.subplots(figsize=(6, 4))

    model_names = [name_converter.get(m, m) for m in model_keys]

    # extract all colors from the .jet map
    cmaplist = [cmap(i) for i in range(cmap.N)]
    # force the first color entry to be grey
    # cmaplist[0] = (.5, .5, .5, 1.0)

    # create the new map
    cmap = matplotlib.colors.LinearSegmentedColormap.from_list(
        'Custom cmap', cmaplist, cmap.N)

    # define the bins and normalize
    bounds = np.logspace(bound, 0., 11)
    norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N)

    image = ax.imshow(matrix, interpolation='none', norm=norm, cmap=cmap)

    # Loop over data dimensions and create text annotations.
    for i in range(len(model_keys)):
        for j in range(len(model_keys)):
            color_float = np.clip(np.abs(matrix[i, j])/ (bound), 0, 1.)
            color_float = (color_float - 0.5)**0.1 + 0.5
            color_float = np.clip(color_float, 0., 1.)
            if not np.isfinite(color_float):
                color_float = 1.

            if matrix[i, j] < lower_bound*150:
                color_float = 0.

            if matrix[i, j] < lower_bound:
                text = '<' + value_str.format(lower_bound)
            else:
                text = value_str.format(matrix[i, j])

            color = '{:.2f}'.format(color_float)
            text = ax.text(
                j, i, text,
                ha="center", va="center", color=color, fontsize=7,
                #path_effects=[pe.Stroke(linewidth=2, foreground='0.'), pe.Normal()],
            )

    cbar = plt.colorbar(image, ax=ax)
    cbar.set_label(r'P-value')
    plt.xticks(range(len(model_keys)), model_names, fontsize=8, rotation=90)
    plt.yticks(range(len(model_keys)), model_names, fontsize=8)
    ax.set_xlabel('Tested Model')
    ax.set_ylabel('Injected Model')
    return fig, ax, cbar

chosen_models = ['fermibubbles_50TeV', 'pi0', 'kra5', 'kra50', 'snr', 'pwn', 'unid', 'combined_stacking']
chosen_models = ['fermibubbles_50TeV', 'pi0', 'kra5', 'kra50', 'snr', 'pwn', 'unid']
matrix_pval, model_keys_pval = get_pval_matrix(model_keys=chosen_models)
fig, ax, cbar = plot_matrix_pval(
    matrix_pval, 
    model_keys=model_keys_pval, 
    name_converter=name_converter,
)

#ax.set_title('Model Confusion Test')
fig.tight_layout()
fig.savefig('{}/model_confusion_matrix_pval.png'.format(plot_dir))
matrix_pval

## Plot NS Bias

In [ ]:
use_poisson = True
if use_poisson:
    allt_i_dict_file = os.path.join(plot_dir, 'allt_i_dict_poisson.pkl')
else:
    allt_i_dict_file = os.path.join(plot_dir, 'allt_i_dict.pkl')

if os.path.exists(allt_i_dict_file):
    print('Loading from file')
    with open(allt_i_dict_file, 'rb') as handle:
        allt_i_dict = pickle.load(handle)
else:
    print('Creating new dict')
    allt_i_dict = {}
    

In [ ]:
ns_bias_plot_keys = ['fermibubbles_50TeV']
ntrials = 200
recalculate = True

for key in ns_bias_plot_keys:
    if key not in allt_i_dict or recalculate:
        ns = int(ns_dict[key])
        max_diff = max(100, int(ns_dict[key] * 1.))
        trial_runner = tr_dict[key]

        n_sigs = np.sort(
            np.r_[np.clip(ns - max_diff, 0., np.inf): ns + max_diff:5],
        ).astype(int)
        print('Submitting {} values for {} from {} +- {}'.format(len(n_sigs), key, ns, max_diff))

        allt_i = get_bias_allt(
            trial_runner, ntrials=ntrials, n_sigs=n_sigs, poisson=use_poisson)

        allt_i_dict[key] = allt_i

with open(allt_i_dict_file, 'wb') as f:
    pickle.dump(allt_i_dict, f, protocol=2)

In [ ]:
for key, allt_i in allt_i_dict.items():
    
    trial_runner = tr_dict[key]
    
    fig, ax = plt.subplots(1, 1)
    plot_ns_bias(ax, trial_runner, allt_i, label=None)
    
    ax.axhline(
        res_dict[key][1], 
        ls='--',
        label='Unblinded ns: {:3.3f}'.format(res_dict[key][1]),
    )
    ax.set(title=r'Analysis: {}'.format(key))
    ax.legend(fontsize=8)
    fig.tight_layout()
    fig.savefig('{}/bias_{}.png'.format(plot_dir, key))
    

In [ ]:
for key, allt_i in allt_i_dict.items():
    
    fig, ax = plt.subplots(1, 1, figsize=(9, 6))
    
    max_diff = 2
    mask = np.abs(allt_i.ns - res_dict[key][1]) < max_diff
    median_ntrue = np.median(allt_i.ntrue[mask])
    mean_ntrue = np.mean(allt_i.ntrue[mask])
    std_ntrue = np.std(allt_i.ntrue[mask])
    ax.hist(
        allt_i.ntrue[mask], color=soft_colors[0], 
        label=r'$P(n_\mathrm{inj}$' + r' | ns = {:3.1f} $\pm$ {:3.1f})'.format(
            res_dict[key][1], max_diff),
        density=True, bins=15,
    )
    
    ax.axvline(
        median_ntrue, color='0.7', ls='--',
        label='Median $n_\mathrm{inj}$' + ': {:3.1f}'.format(median_ntrue),
    )
    
    ax.axvline(
        res_dict[key][1], color=soft_colors[1], ls='--',
        label='Unblinded ns: {:3.1f}'.format(res_dict[key][1]),
    )
    ax.set(title=r'Analysis: {}'.format(key) + ' | $n_\mathrm{inj}$' + '(ns={:3.1f}) = {:3.1f} $\pm$ {:3.1f} [$\mu \pm \sigma$ ]'.format(
        res_dict[key][1], mean_ntrue, std_ntrue))
    ax.set_xlabel('True $n_\mathrm{inj}$')
    ax.set_ylabel('PDF')
    ax.legend()
    fig.tight_layout()
    fig.savefig('{}/bias_at_unblinded_ns_{}.png'.format(plot_dir, key))
    

# Scratch Area

#### Compute bkg trial significance

In [ ]:
p_val_dict_bkg = {}
sigma_dict_bkg = {}
max_n = 10000000
for key, bg in bkg_dict.items():
    print('key:', key)
    bg_tsd = cy.dists.TSD(bg[:max_n])
    p_val_dict_bkg[key] = bg_tsd.sf(bg[:max_n])
    sigma_dict_bkg[key] = bg_tsd.sf_nsigma(bg[:max_n])


#### Plot Trial Correlation

In [ ]:
mask = np.zeros_like(bkg_dict['snr'][:max_n])
sigma_threshold = -10.5

for key, tr in sigma_dict_bkg.items():

    mask = np.logical_or(mask, sigma_dict_bkg[key] > sigma_threshold)
    

In [ ]:
import matplotlib as mpl

def plot_corr_ax(ax, key1, key2, mask=None, norm=None):
    
    if mask is None:
        mask = np.ones_like(sigma_dict_bkg[key1], dtype=bool)
        
    ax.hist2d(
        sigma_dict_bkg[key1][mask], sigma_dict_bkg[key2][mask],
        bins=bins, norm=norm, cmin=1,
    )
    ax.plot(
        (bins[0][0], bins[0][-1]), (bins[0][0], bins[0][-1]), 
        ls='--', color='0.7', lw=3,
    )
    ax.set_xlabel('$n\cdot \sigma$ of {}'.format(key1))
    ax.set_ylabel('$n\cdot \sigma$ of {}'.format(key2))

fig, axes = plt.subplots(3, 1, figsize=(9, 9))

bins = (np.linspace(0, 6), np.linspace(0, 6))

mask = None
plot_corr_ax(axes[0], 'snr', 'pwn', mask=mask)
plot_corr_ax(axes[1], 'snr', 'unid', mask=mask)
plot_corr_ax(axes[2], 'unid', 'pwn', mask=mask)
fig.tight_layout()


In [ ]:
from scipy import stats

corr_keys = ['snr', 'pwn', 'unid']
corr_keys = ['pi0', 'kra5', 'kra50']
#corr_keys = ['pi0', 'kra5', 'kra50', 'snr', 'pwn', 'unid']

max_nsigma = np.max(
    np.stack([sigma_dict_bkg[k] for k in corr_keys]),
    axis=0,
)

bg_max = cy.dists.TSD(max_nsigma)
bins = np.linspace(0, 6, 100)
fig, ax = plot_bkg_trials(bg_max, bins=bins, color=soft_colors[1])
ax.hist(max_nsigma, bins=bins)
ax.set_xlabel('Max n-sigma')
ax.set_yscale('log')

nsigma_chosen = 4.705
pval_chosen = stats.norm.sf(nsigma_chosen)
nsigma_corrected = bg_max.sf_nsigma(nsigma_chosen)

pval_corrected = bg_max.sf(nsigma_chosen)
print('Correcting for: {}'.format(corr_keys))
print('Pre-trial N-sigma of: {}'.format(nsigma_chosen))
print('Post-trial correlated: {} | factor: {}'.format(nsigma_corrected, pval_corrected/pval_chosen))
print('Post-trial conservative: {} | factor: {}'.format(stats.norm.isf(pval_chosen * len(corr_keys)), len(corr_keys)))
